# SI 330: Data Manipulation 
## 06 - Enhancing pandas peformance

### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

## Learning Objectives

* explain why we should be concerned with algorithmic efficiency
* be able to describe 5 types of Big O notation
* be able to report execution times in Jupyter
* be able to explain profiling output in Jupyter

**NOTE: You only need to conda install -y line_profiler ONCE!**

In [ ]:
!conda install -y line_profiler

## Big O Notation

* "order of growth"
* a way to categorize how algorithms behave with increasing data size
* commonly used in Computer Science

https://rob-bell.net/2009/06/a-beginners-guide-to-big-o-notation/

### $O(1)$

In [ ]:
def IsFirstElementNull(aList):
    return aList[0] == None

### $O(n)$

In [ ]:
def ContainsValue(aList, aValue):
    for e in aList:
        if e == aValue:
            return True
    return False

### $O(n^2$)

In [ ]:
def ContainsDuplicates(aList):
    for i in range(len(aList)):
        for j in range(len(aList)):
            if i == j:
                continue
            if aList[i] == aList[j]:
                return True
    return False

### $O(2^n$)

In [ ]:
def Fibonacci(n):
    if (n <= 1):
        return n
    return Fibonacci(n - 2) + Fibonacci(n -1 )

### $O(log(n))$

In each iteration, we are halfway closer to finishing the problem.

https://stackoverflow.com/questions/2307283/what-does-olog-n-mean-exactly

This is why, for example, looking up people in a phone book is O(log n). You don't need to check every person in the phone book to find the right one; instead, you can simply divide-and-conquer by looking based on where their name is alphabetically, and in every section you only need to explore a subset of each section before you eventually find someone's phone number.

Of course, a bigger phone book will still take you a longer time, but it won't grow as quickly as the proportional increase in the additional size.

We can expand the phone book example to compare other kinds of operations and their running time. We will assume our phone book has businesses (the "Yellow Pages") which have unique names and people (the "White Pages") which may not have unique names. A phone number is assigned to at most one person or business. We will also assume that it takes constant time to flip to a specific page.

Here are the running times of some operations we might perform on the phone book, from best to worst:

O(1) (best case): Given the page that a business's name is on and the business name, find the phone number.

O(1) (average case): Given the page that a person's name is on and their name, find the phone number.

O(log n): Given a person's name, find the phone number by picking a random point about halfway through the part of the book you haven't searched yet, then checking to see whether the person's name is at that point. Then repeat the process about halfway through the part of the book where the person's name lies. (This is a binary search for a person's name.)

O(n): Find all people whose phone numbers contain the digit "5".

O(n): Given a phone number, find the person or business with that number.

O(n log n): There was a mix-up at the printer's office, and our phone book had all its pages inserted in a random order. Fix the ordering so that it's correct by looking at the first name on each page and then putting that page in the appropriate spot in a new, empty phone book.

For the below examples, we're now at the printer's office. Phone books are waiting to be mailed to each resident or business, and there's a sticker on each phone book identifying where it should be mailed to. Every person or business gets one phone book.

O(n log n): We want to personalize the phone book, so we're going to find each person or business's name in their designated copy, then circle their name in the book and write a short thank-you note for their patronage.

O(n2): A mistake occurred at the office, and every entry in each of the phone books has an extra "0" at the end of the phone number. Take some white-out and remove each zero.

O(n · n!): We're ready to load the phonebooks onto the shipping dock. Unfortunately, the robot that was supposed to load the books has gone haywire: it's putting the books onto the truck in a random order! Even worse, it loads all the books onto the truck, then checks to see if they're in the right order, and if not, it unloads them and starts over. (This is the dreaded bogo sort.)

O(nn): You fix the robot so that it's loading things correctly. The next day, one of your co-workers plays a prank on you and wires the loading dock robot to the automated printing systems. Every time the robot goes to load an original book, the factory printer makes a duplicate run of all the phonebooks! Fortunately, the robot's bug-detection systems are sophisticated enough that the robot doesn't try printing even more copies when it encounters a duplicate book for loading, but it still has to load every original and duplicate book that's been printed.

For more mathematical explanation you can checkout how the time complexity arrives to log n here. https://hackernoon.com/what-does-the-time-complexity-o-log-n-actually-mean-45f94bb5bfbf

## Timing and Profiling in Jupyter

The following code and examples are from [Sofia Heisler's repo](https://github.com/s-heisler/pycon2017-optimizing-pandas.git).
It is available at https://github.com/umsi-data-science/s-heisler-pycon2017-optimizing-pandas

In [ ]:
import pandas as pd
import numpy as np
from math import *

### Read in the data

In [ ]:
df = pd.read_csv('data/new_york_hotels.csv', encoding='cp1252')

In [ ]:
df.head()

In [ ]:
df.columns

## Benchmarking example

#### Define the normalization function

In [ ]:
def normalize(df, pd_series):
    pd_series = pd_series.astype(float)
    #print(pd_series)
    # Find upper and lower bound for outliers
    avg = np.mean(pd_series)
    sd  = np.std(pd_series)
    #print(avg,sd)
    lower_bound = avg - 2*sd
    upper_bound = avg + 2*sd

    # Collapse in the outliers
    df.loc[pd_series < lower_bound , "high_rate" ] = lower_bound
    df.loc[pd_series > upper_bound , "high_rate" ] = upper_bound
    # Finally, take the log (why add 1?)
    normalized_price = np.log(df["high_rate"].astype(float)+1)
    return normalized_price

In [ ]:
normalize(df, df['high_rate'])

#### Timing the normalization function

In [ ]:
%timeit df['high_rate_normalized'] = normalize(df, df['high_rate'])

#### Profiling the normalization function

In [ ]:
    %load_ext line_profiler

In [ ]:
%lprun -f normalize df['high_rate_normalized'] = normalize(df, df['high_rate'])

### <font color="magenta"> Q1 (2 points): Where is most of the time being spent?

Insert answer here.

### <font color="magenta"> Q2 (2 points): What order is the normalize function being executed?  How can you figure this out?

Insert answer here.

## There are at least 5 different ways to operate on a pandas Series:
* iterate with a for loop
* use iterrows
* use apply
* use vectorized functions
* use cython

## Haversine definition

https://en.wikipedia.org/wiki/Haversine_formula

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    miles_constant = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    mi = miles_constant * c
    return mi

## Iterating using for and loc

### <font color="magenta"> Q3 (2 points): Write pythonic code that iterates over each row to calculate the distance from lat1= 40.671, lon1= -73.985 to each hotel's latitude and longitude.

In [ ]:
# insert code here 

## Iterrows Haversine

In [ ]:
%%timeit
# Haversine applied on rows via iteration
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(haversine(40.671, -73.985,\
                                      row['latitude'], row['longitude']))
df['distance'] = haversine_series

## Apply Haversine on rows

### Timing "apply"

In [ ]:
%timeit df['distance'] =\
df.apply(lambda row: haversine(40.671, -73.985,\
                               row['latitude'], row['longitude']), axis=1)

#### Profiling "apply"

In [ ]:
# Haversine applied on rows
%lprun -f haversine \
df.apply(lambda row: haversine(40.671, -73.985,\
                               row['latitude'], row['longitude']), axis=1)

## Vectorized implementation of Haversine applied on Pandas series

#### Timing vectorized implementation

In [ ]:
# Vectorized implementation of Haversine applied on Pandas series
%timeit df['distance'] = haversine(40.671, -73.985,\
                                   df['latitude'], df['longitude'])

#### Profiling vectorized implementation

In [ ]:
# Vectorized implementation profile
%lprun -f haversine haversine(40.671, -73.985,\
                              df['latitude'], df['longitude'])

## Vectorized implementation of Haversine applied on NumPy arrays

#### Timing vectorized implementation

In [ ]:
# Vectorized implementation of Haversine applied on NumPy arrays
%timeit df['distance'] = haversine(40.671, -73.985,\
                         df['latitude'].values, df['longitude'].values)

In [ ]:
%%timeit
# Convert pandas arrays to NumPy ndarrays
np_lat = df['latitude'].values
np_lon = df['longitude'].values

#### Profiling vectorized implementation

In [ ]:
%lprun -f haversine df['distance'] = haversine(40.671, -73.985,\
                        df['latitude'].values, df['longitude'].values)

### <font color="magenta"> Q4 (2 points): Record the times above and comment on how efficiency improves going from the first implementation to the last

Insert answer here.

## Not good enough?  Cythonize that loop

#### Load the cython extension

In [ ]:
%load_ext cython

#### Run unaltered Haversine through Cython

In [ ]:
%%cython -a

# Haversine cythonized (no other edits)
import numpy as np
cpdef haversine_cy(lat1, lon1, lat2, lon2):
    miles_constant = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    mi = miles_constant * c
    return mi

#### Time it

In [ ]:
%timeit df['distance'] =\
       df.apply(lambda row: haversine_cy(40.671, -73.985,\
                row['latitude'], row['longitude']), axis=1)

### <font color="magenta"> Q5 (2 points): How much more efficient is the cython version compared to the best non-cython version from above?

Insert Answer Here.

#### Redefine Haversine with data types and C libraries

### FYI Only

In [ ]:
%%cython -a
# Haversine cythonized
from libc.math cimport sin, cos, acos, asin, sqrt

cdef deg2rad_cy(float deg):
    cdef float rad
    rad = 0.01745329252*deg
    return rad
    
cpdef haversine_cy_dtyped(float lat1, float lon1, float lat2, float lon2):
    cdef: 
        float dlon
        float dlat
        float a
        float c
        float mi
    
    lat1, lon1, lat2, lon2 = map(deg2rad_cy, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    mi = 3959 * c
    return mi


#### Time it

In [ ]:
%timeit df['distance'] =\
df.apply(lambda row: haversine_cy_dtyped(40.671, -73.985,\
                              row['latitude'], row['longitude']), axis=1)

# END OF NOTEBOOK
Please remember to submit your notebook in .ipynb and .html formats.